# 1.1 Dataset Preparation

In [93]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
digits = load_digits()
data = digits [ "data" ]
images = digits [ "images" ]
target = digits [ "target" ]
target_names = digits [ "target_names" ]


In [94]:
# Subtract mean vector and divide each feature vector by the standard deviation of the whole dataset
X = (data - np.mean(data,axis=0))
X = X/np.std(X)

X_k = np.zeros((10,174,64))
Y_k = np.zeros((10,174))
for i in range(0,10):
    xk = data[np.where(target==i)]
    X_k[i] = xk[0:174,:]
    Y_k[i] = np.ones((1,174))*i

# 1.2 One vs Rest

In [95]:
rfc = RandomForestClassifier(n_estimators = 10, class_weight = "auto")

rfc.fit(X_k,Y_k)

ValueError: Found array with dim 3. Estimator expected <= 2.

In [15]:
np.mean(data,axis=0)

array([  0.00000000e+00,   3.03839733e-01,   5.20478575e+00,
         1.18358375e+01,   1.18480801e+01,   5.78185865e+00,
         1.36227045e+00,   1.29660545e-01,   5.56483027e-03,
         1.99387869e+00,   1.03823038e+01,   1.19794101e+01,
         1.02793545e+01,   8.17584864e+00,   1.84641068e+00,
         1.07957707e-01,   2.78241514e-03,   2.60155815e+00,
         9.90317195e+00,   6.99276572e+00,   7.09794101e+00,
         7.80634391e+00,   1.78853645e+00,   5.00834725e-02,
         1.11296605e-03,   2.46967168e+00,   9.09126322e+00,
         8.82136895e+00,   9.92710072e+00,   7.55147468e+00,
         2.31775181e+00,   2.22593211e-03,   0.00000000e+00,
         2.33945465e+00,   7.66722315e+00,   9.07178631e+00,
         1.03016138e+01,   8.74401781e+00,   2.90929327e+00,
         0.00000000e+00,   8.90372844e-03,   1.58375070e+00,
         6.88146912e+00,   7.22815804e+00,   7.67223150e+00,
         8.23650529e+00,   3.45631608e+00,   2.72676683e-02,
         7.23427935e-03,